# SQLite3 Connection

In [1]:
import sqlite3
con = sqlite3.connect('CSCI4333_Sailor.db')
cursor = con.cursor()

# Functions to print table info

In [7]:
def print_table(table_name="Sailors"):
    res_table = cursor.execute("SELECT * FROM "+table_name)
    attribute = [x[0] for x in res_table.description]
    print(attribute)
    print('--------------')
    for row in res_table: print(row)

def print_query_res(res_table):
    #res_table = cursor.execute("SELECT * FROM "+table_name)
    attribute = [x[0] for x in res_table.description]
    print(attribute)
    print('--------------')
    for row in res_table: print(row)

In [3]:
print_table("Reserves")

['sid', 'bid', 'day']
--------------
('22', '101', '10/10/04')
('22', '102', '10/10/04')
('22', '103', '10/8/04')
('22', '104', '10/7/04')
('31', '102', '11/10/04')
('31', '103', '11/6/04')
('31', '104', '11/12/04')
('64', '101', '9/5/04')
('64', '102', '9/8/04')
('74', '103', '9/8/04')


In [4]:
print_table("Boats")

['bid', 'bname', 'color']
--------------
('101', 'Interlake', 'blue')
('102', 'Interlake', 'red')
('103', 'Clipper', 'green')
('104', 'Marine', 'red')


In [5]:
print_table("Sailors")

['sid', 'sname', 'rating', 'age']
--------------
('22', 'Dustin', 7, 45)
('29', 'Brutus', 1, 35)
('31', 'Lubber', 8, 55)
('32', 'Andy', 8, 25)
('58', 'Rusty', 10, 35)
('64', 'Horatio', 7, 35)
('71', 'Zorda', 10, 16)
('74', 'Horatio', 9, 35)
('85', 'Art', 3, 25)
('95', 'Bob', 3, 63)


# Simple Queries

Q1: Find the name of the Sailor who reserved boat 103?

In [10]:
#Solution 1: Basic Query
statement = "SELECT sname FROM Sailors S,Reserves R WHERE S.sid=R.sid AND R.bid='103'"
res_table = cursor.execute(statement)
print_query_res(res_table)

['sname']
--------------
('Dustin',)
('Lubber',)
('Horatio',)


In [12]:
#Solution 2: Nested Query
statement = '''
SELECT sname FROM Sailors S 
WHERE S.sid IN (SELECT R.sid FROM Reserves R WHERE R.bid='103')
'''

res_table = cursor.execute(statement)
print_query_res(res_table)

['sname']
--------------
('Dustin',)
('Lubber',)
('Horatio',)


In [14]:
#Solution 3: Natural Join Solution
statement = "SELECT sname FROM Sailors S NATURAL JOIN Reserves R WHERE R.bid='103'"

res_table = cursor.execute(statement)
print_query_res(res_table)

['sname']
--------------
('Dustin',)
('Lubber',)
('Horatio',)


In [15]:
#Solution 4: Inner Join Solution
statement = "SELECT sname FROM Sailors S INNER JOIN Reserves R ON S.sid=R.sid WHERE R.bid='103'"

res_table = cursor.execute(statement)
print_query_res(res_table)

['sname']
--------------
('Dustin',)
('Lubber',)
('Horatio',)


In [18]:
#Solution 5: Correlated Nested Query
statement = '''
SELECT sname FROM Sailors S 
WHERE EXISTS (SELECT * FROM Reserves R WHERE R.bid='103' AND R.sid=S.sid)
'''

res_table = cursor.execute(statement)
print_query_res(res_table)

['sname']
--------------
('Dustin',)
('Lubber',)
('Horatio',)


# Aggregation Query

Q2: Find the maximal rating of the sailors whose age is greater than 18

In [19]:
statement = '''
SELECT MAX(S.rating) FROM Sailors S 
WHERE S.age>18
'''

res_table = cursor.execute(statement)
print_query_res(res_table)

['MAX(S.rating)']
--------------
(10,)


Q3: Find the mimimal age for each rating stored in the database. You answer should contains (rating, age)

In [23]:
statement = '''
SELECT S.rating, MIN(S.age) 
FROM Sailors S
GROUP BY S.rating
'''

res_table = cursor.execute(statement)
print_query_res(res_table)

['rating', 'MIN(S.age)']
--------------
(1, 35)
(3, 25)
(7, 35)
(8, 25)
(9, 35)
(10, 16)


Q4: Find the mimimal age for each rating stored in the database. We only consider the sailor whose age is greater or equal to 18. You answer should contains (rating, age)

In [22]:
statement = '''
SELECT S.rating, MIN(S.age) 
FROM Sailors S
WHERE S.age>=18 
GROUP BY S.rating
'''

res_table = cursor.execute(statement)
print_query_res(res_table)

['rating', 'MIN(S.age)']
--------------
(1, 35)
(3, 25)
(7, 35)
(8, 25)
(9, 35)
(10, 35)


Q5: for each rating that has at least two sailors, find the mimimal age for each rating. We only consider the sailor whose age is greater or equal to 18. You answer should contains (rating, age)

In [25]:
statement = '''
SELECT S.rating, MIN(S.age) 
FROM Sailors S
WHERE S.age>=18 
GROUP BY S.rating
HAVING COUNT(*)>1
'''

res_table = cursor.execute(statement)
print_query_res(res_table)

['rating', 'MIN(S.age)']
--------------
(3, 25)
(7, 35)
(8, 25)


Q6: Find the average rating for each age stored in the database. You answer should contains (age, rating)

In [27]:
#For every age of sailors, find the average rating
statement = '''
SELECT S.age, AVG(S.rating) 
FROM Sailors S
GROUP BY S.age
'''

res_table = cursor.execute(statement)
print_query_res(res_table)

['age', 'AVG(S.rating)']
--------------
(16, 10.0)
(25, 5.5)
(35, 6.75)
(45, 7.0)
(55, 8.0)
(63, 3.0)


Q7: Find the count of boats for each boat color stored in the database. You answer should contains (color, count)

In [31]:


statement = '''
SELECT B.color, COUNT(*) 
FROM Boats B
GROUP BY B.color
'''

res_table = cursor.execute(statement)
print_query_res(res_table)



['color', 'COUNT(*)']
--------------
('blue', 1)
('green', 1)
('red', 2)


Q8: Find the count of reservations for each boat color stored in the database. You answer should contains (color, count)

In [34]:
#For every color of the boat, count how many times the sailors reserved the boats

statement = '''
SELECT B.color,COUNT(sid)
FROM Boats B, Reserves R
WHERE B.bid = R.bid
GROUP BY B.color
'''

res_table = cursor.execute(statement)
print_query_res(res_table)



['color', 'COUNT(sid)']
--------------
('blue', 2)
('green', 3)
('red', 5)


Q9: Find the count of unique sailors for each boat color stored in the database. You answer should contains (color, count)

In [36]:
#For every color of the boat, count how many unique sailors who reserved the boats

statement = '''
SELECT B.color,COUNT(DISTINCT sid)
FROM Boats B, Reserves R
WHERE B.bid = R.bid
GROUP BY B.color
'''

res_table = cursor.execute(statement)
print_query_res(res_table)


['color', 'COUNT(DISTINCT sid)']
--------------
('blue', 2)
('green', 3)
('red', 3)


Q10: Find the count of unique sailors for each boat color stored in the database. You answer should contains (color, count)

In [ ]:
#For every color of the boat, count how many unique sailors who reserved the boats

statement = '''
SELECT B.color,COUNT(DISTINCT sid)
FROM Boats B, Reserves R
WHERE B.bid = R.bid
GROUP BY B.color
'''

res_table = cursor.execute(statement)
print_query_res(res_table)


Q11: Find the average age of the reservations for each boat stored in the database. You answer should contains (bid, age)

In [38]:
statement = '''
SELECT R.bid, AVG(S.age)
FROM Reserves R, Sailors S
WHERE S.sid=R.sid
GROUP BY R.bid
'''
res_table = cursor.execute(statement)
print_query_res(res_table)

['bid', 'AVG(S.age)']
--------------
('101', 40.0)
('102', 45.0)
('103', 45.0)
('104', 50.0)


Q12: For each boat which reserved by at least three unique sailors, find the average age of the reservations for each boat stored in the database. You answer should contains (bid, age)

In [40]:
statement = '''
SELECT R.bid, AVG(S.age)
FROM Reserves R, Sailors S
WHERE S.sid=R.sid
GROUP BY R.bid
HAVING COUNT(DISTINCT S.sid)>=3
'''
res_table = cursor.execute(statement)
print_query_res(res_table)

['bid', 'AVG(S.age)']
--------------
('102', 45.0)
('103', 45.0)


Q13: For each boat which reserved by at least two unique sailors that rating are greater than 5, find the average age of the reservations for each boat stored in the database. You answer should contains (bid, age)

In [46]:
statement = '''
SELECT R.bid, AVG(S.age)
FROM Reserves R, Sailors S
WHERE S.sid=R.sid AND S.rating>5
GROUP BY R.bid
HAVING COUNT(DISTINCT S.sid)>=2
'''
res_table = cursor.execute(statement)
print_query_res(res_table)

['bid', 'AVG(S.age)']
--------------
('101', 40.0)
('102', 45.0)
('103', 45.0)
('104', 50.0)


Q13: For each boat which reserved by at least one sailor whose age is less than 35, find the average age of the reservations for each boat stored in the database. You answer should contains (bid, age)

In [48]:
#For each boat (bid) which reserved by at least one sailor 
#whose age is less than 20,find the average rating of the 
#sailors who reserved it

statement = '''
SELECT R.bid, AVG(S.age)
FROM Reserves R, Sailors S
WHERE S.sid=R.sid
GROUP BY R.bid
HAVING MIN(S.age)<=35
'''
res_table = cursor.execute(statement)
print_query_res(res_table)

['bid', 'AVG(S.age)']
--------------
('101', 40.0)
('102', 45.0)
('103', 45.0)


Q14: Find the name for the sailors who reserved at least two boats.

In [53]:
statement = '''
SELECT S.sname
FROM Sailors S, Reserves R
WHERE S.sid=R.sid
GROUP BY S.sid, S.sname
HAVING COUNT(bid)>=2
'''

res_table = cursor.execute(statement)
print_query_res(res_table)



['sname']
--------------
('Dustin',)
('Lubber',)
('Horatio',)


# Aggregation + Nested Query 

Q15: Find the name for the sailors who reserved at least two boats. (Using nested query)

In [55]:
statement = '''
SELECT S.sname
FROM Sailors S
GROUP BY S.sid, S.sname
HAVING (SELECT COUNT(bid) FROM Reserves R WHERE S.sid=R.sid)>=2
'''

res_table = cursor.execute(statement)
print_query_res(res_table)



['sname']
--------------
('Dustin',)
('Lubber',)
('Horatio',)


Q16: Find the name of sailors who reserved at least two times red boats (Using nested query)

In [57]:
statement = '''
SELECT S.sname
FROM Sailors S
GROUP BY S.sid, S.sname
HAVING (SELECT COUNT(bid) 
        FROM Reserves R 
        WHERE S.sid=R.sid AND
        R.bid IN (
                   SELECT B.bid FROM Boats B WHERE B.color='red'
                 )
       )>=2
'''

res_table = cursor.execute(statement)
print_query_res(res_table)



['sname']
--------------
('Dustin',)
('Lubber',)


Q17: For each boat (bid) which reserved by at least two unique sailors, find the average rating of the sailors who reserved it. Note: We count each sailor only once

In [66]:
#For each boat (bid) which reserved by at least two sailor 
#find the average rating of the sailors who reserved it. 
#Note: We count each sailor only once!

statement = '''
SELECT T.sid, AVG(T.rating) 
FROM  (SELECT DISTINCT S.sid,R.bid,S.rating
       FROM Reserves R, Sailors S
       WHERE R.sid=S.sid) AS T
GROUP BY T.sid
HAVING COUNT(sid)>=2
'''

res_table = cursor.execute(statement)
print_query_res(res_table)




['sid', 'AVG(T.rating)']
--------------
('22', 7.0)
('31', 8.0)
('64', 7.0)


Q18: Return the name of sailor(s) who reserved the highest number of boats

In [68]:
statement = '''
SELECT sname 
FROM Sailors S, Reserves R
WHERE S.sid = R.sid 
GROUP BY S.sid, S.sname
HAVING COUNT(*)=(SELECT MAX(T.c) 
                 FROM (SELECT R2.sid, COUNT(*) AS c 
                       FROM Reserves R2 
                       GROUP BY R2.sid) AS T
                 )
'''


res_table = cursor.execute(statement)
print_query_res(res_table)


['sname']
--------------
('Dustin',)
